```
python -m drivingppo.simsim
```

In [ ]:
# 학습된 모델 확인

import time, json
from typing import Literal, Callable
import random
from random import randint

from drivingppo.common import set_seed, DIS_SCFAC, WORLD_DT, ACTION_REPEAT, LOOKAHEAD_POINTS
from drivingppo.world import World, Car, OBSTACLE_VALUE, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.environment import WorldEnv
from drivingppo.model import run

import numpy as np

from stable_baselines3 import PPO

LOG_DIR = "./logs/"
CHECKPOINT_DIR = './checks/'

from world_samples import NEAR, W_CONFIG
from world_samples import gen_from, gen_1t, gen_2t, gen_3t, gen_3l, gen_obs
from world_samples import (
    generate_random_world_plain,
    generate_random_world_obs_matrix,
    generate_random_world_narrow,
    generate_random_world_obs_between,
    generate_world_simpleLine,
    generate_world_square,
    generate_world_zipper,
)

import math
pi = math.pi
pi2 = pi*2


MAP_W, MAP_H = 150, 150

def tw():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 40),
            (20, 40),
            (50, 60),
            (60, 50),

            # (10, 50),
            # (30, 70),
            # (70, 50),
            # (90, 70),

            # (10, 30),
            # (10, 50),
            # (10, 70),
            # (40, 70),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w

def gen_3l(): return generate_random_world_plain(map_h=300, map_w=300, num=10,               min_dist=NEAR*2, max_dist=DIS_SCFAC, ang_lim=pi*1.0, ang_init='p',    spd_init=0.0)  # 테스트용: 같은 패턴인데 좀 긴 경로이고 초기 0

name = 'wwwwwwwwwwwwwwwwwwwwwwwwww'

# set_seed(10)
# run(gen_3t, name, auto_close_at_end=False)
# run(gen_3l, name, auto_close_at_end=False)
run(tw, name, auto_close_at_end=False)
# run(gen_from(gen_3l, 0, 1), name, auto_close_at_end=False)
# run(tw, f'wwwwwwwwwwwwwwwwwwwwwwwwww', auto_close_at_end=False)

print('#########')

In [ ]:
a = [
]
a.sort()
a

In [ ]:
# 각 모델 
from drivingppo.model import evaluate
from drivingppo.common import set_seed
# from world_samples import generate_random_world_plain
from world_samples import gen_3l, generate_random_world_plain, NEAR, DIS_SCFAC
import math; pi = math.pi

# def gen_3l(): return generate_random_world_plain(map_h=300, map_w=300, num=10,               min_dist=NEAR*2, max_dist=DIS_SCFAC, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')  # 테스트용: 같은 패턴인데 좀 긴 경로

# name = 'wwwwwwwwwwwwwwwwwwwwwwwwww'
# set_seed(0)
# a = evaluate(name, gen_3l, 100,
#              csv_path='results/'+name+'.csv'
#              )

def a(name0:str):
    for seed in range(100, 501, 100):
        name = f'{name0}-{seed}'
        print(f'\n\nNAME: {name}')

        set_seed(0)
        evaluate(name, gen_3l, 100,
                    csv_path='results/'+name+'.csv'
                    )

a('wwwwwwwwwwwwwwwwwwwwwwwwww')

In [ ]:
import os, glob
from pathlib import Path

import pandas as pd

# TODO configId 맞춰
def aggregate_csv_data__old(directory_path):
    # 디렉토리 내 모든 CSV 파일 경로 탐색
    search_pattern = os.path.join(directory_path, '**', '*.csv')
    csv_files = glob.glob(search_pattern, recursive=True)
    
    if not csv_files:
        raise ValueError(f"'{directory_path}' 경로에 CSV 파일이 존재하지 않습니다.")

    processed_rows = []

    for file_path in csv_files:
        id = str(Path(os.path.basename(file_path)).with_suffix(''))
        idParts = id.split('-')
        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            print(f"!!!!!!!! 읽기 실패: {file_path} ... {e}")
            continue

        N     = 4  if idParts[0][0] == 'y'  else 20
        G     = idParts[0][1:]  if len(idParts[0][1:]) > 0  else 100
        Coord = 'ACC'  if idParts[1] == 'fagnt'  else 'SRC'
        FE    = 'I'  if idParts[2] == 'nofe'  else 'WSWE'  if idParts[2] == 'shwp'  else 'VMLP'  if idParts[2] == 'vmlp'  else idParts[2]
        seed  = idParts[3]
        S = ''
        configId = f'N{N}-G{G}-{Coord}-{FE}{S}'
        row_data:dict[str, float|str] = {
            'id':    id,
            'configId': configId,
            'N':     20,
            'gamma': G,
            'Coord': Coord,
            'FE':    FE,
            'seed':  seed,
        }

        # 수 컬럼: 평균 계산
        numeric_cols = df.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            row_data[col] = df[col].mean()

        # 텍스트 컬럼: 값별 비율 계산
        categorical_cols = df.select_dtypes(exclude=['number']).columns
        for col in categorical_cols:
            proportions = df[col].value_counts(normalize=True, dropna=False)
            for category_val, proportion in proportions.items():
                column_name = f"{col}/{category_val}"
                row_data[column_name] = proportion

        processed_rows.append(row_data)

    df = pd.DataFrame(processed_rows)
    df = df.fillna(0)  # 특정 파일에서만 있는 범주가 다른 파일에서는 NaN 처리되므로 0으로 채움.

    return df

df = aggregate_csv_data__old('./results')
df.to_csv('./results.csv', index=False)

In [ ]:
a = 0.95
a**50

In [ ]:
# 맵 생성 확인
import math
pi = math.pi

from drivingppo.common import set_seed
from drivingppo.world import create_empty_map
from drivingppo.simsim import WorldController, World, Car
from world_samples import gen_1t, gen_2t, gen_3t, gen_3l, generate_random_world_plain
import numpy as np
import math, random
from random import randint
from drivingppo.world import World, Car, OBSTACLE_VALUE, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.environment import SPD_SCFAC

NEAR = 3.0
W_CONFIG = {
    'near': NEAR,
    'far': 35.0,
}

set_seed(0)
world = gen_3t(20)

app = WorldController(
    world,
    time_accel=2,
    use_real_time=False,
    frame_delay=33)

In [ ]:
# 모델 1회 실행 결과 확인
from drivingppo.environment import WorldEnv, distance_score_near, distance_score_far

import numpy as np
from stable_baselines3 import PPO

env = WorldEnv(
    world_generator=lambda: World(),
    render_mode='debug',
    auto_close_at_end=False
)
model = PPO.load('./ppo_world_checkpoints/ppo.zip', env=env)

path_data = []

# agent_speed, path_data, obs_near, obs_nearest_angle, obs_nearest_near
np.array([0.0] + path_data, dtype=np.float32)

# ppo 로그 파일(./logs/ppo.csv)에서 obs 부분만 여기 복붙하여 확인
obs = np.array([+8.321,+0.057,+0.091,+0.996,+0.056,+0.082,+0.997,+0.056,+0.073,+0.997,+0.055,+0.064,+0.998,+0.054,+0.055,+0.998,+0.406,+0.714,+0.689,+0.657,+0.616,+0.562,+0.491,+0.394,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000,+0.000
])

action, _ = model.predict(obs, deterministic=True)

print(action)

In [ ]:
# WorldController 켜기
import math

from drivingppo.world import World, Car, create_empty_map, angle_of, distance_of, rad_to_deg
from drivingppo.common import MAP_W, MAP_H, LOOKAHEAD_POINTS
from drivingppo.environment import distance_score_near, distance_score_far, apply_action, observation_str, action_str
from drivingppo.simsim import WorldController

pi = math.pi

player = Car({
    'playerPos': {'x': 50, 'z': 10},
    "playerSpeed": 10,
    "playerBodyX": 0,
    "playerBodyY": 0,
    "playerBodyZ": 0,
    "playerHealth": 0,
    "playerTurretX": 0,
    "playerTurretY": 0,
})

# obstacle_map, w, h = load_obstacle_map('./map-50.txt')

waypoints = [(10, 100), (100, 150), (150, 100), (200, 200), (250, 250)]
waypoints = []

world = World(
    player=player,
    wh=(100, 100),
#   obstacle_map=obstacle_map,
    waypoints=waypoints,
    config={
        'angle_start': -pi/4,
        'angle_end': pi/4
})

WorldController(
    world,
    time_accel=1,
    use_real_time=False,
    frame_delay=33,
    use_stop=False,
    config={
        'TrackingMode': False,
        'LogMode': False,
        'api_delay': 1000,
    })

In [ ]:
# 거리점수 확인

import math
import numpy as np
import matplotlib.pyplot as plt
from drivingppo.environment import distance_score_near
from drivingppo.environment import distance_score_far

# def distance_score_near(x:float) -> float:
#     d = x + 10.0
#     x = 100./d/d
#     if x <= 1:
#         return x
#     else:
#         return 1.0

# def distance_score_far(distance:float) -> float:
#     return math.log(distance + 1.0)/10.0

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

xs = np.array(range(3000))/100
nears = np.array([distance_score_near(x) for x in xs])
fars  = np.array([distance_score_far(x)  for x in xs])

print('NEAR')
for x in range(0, 21):
    print(f'{x}: {distance_score_near(x):.4f}')

print('FAR')
for x in range(0, 21):
    print(f'{x}: {distance_score_far(x):.4f}')

axs[0].plot(xs, nears)
axs[0].grid(True)
axs[0].set_yticks([0.0, 0.5, 1.0])
axs[0].set_xticks([0, 1, 2, 10, 20, 30])

axs[1].plot(xs, fars)
axs[1].grid(True)
# axs[1].set_yticks([0.0, 0.5, 1.0])
axs[1].set_xticks([0, 1, 10, 20, 30, 40])

plt.show()

In [ ]:
# 랜덤 각도 분포 보기
from world_samples import get_random_angle

import matplotlib.pyplot as plt
import numpy as np

a = pi * 1.0
sigma = a/pi*2
data = [get_random_angle(a) for _ in range(5000)]

plt.figure(figsize=(10, 6))
count, bins, ignored = plt.hist(data, bins=50, density=True,)

pdf = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * (bins / sigma)**2)
plt.plot(bins, pdf, linewidth=2, color='r', label='PDF (Theoretical)')

plt.title('Histogram of Normal Distribution')
plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.legend()
plt.grid(True)
plt.show()